In [5]:
#file variables

#pretrained_bert = 'allenai/scibert_scivocab_uncased'
#pretrained_bert = 'dmis-lab/biobert-base-cased-v1.2'
pretrained_bert = 'cimm-kzn/endr-bert'
#pretrained_bert = 'SpanBERT/spanbert-base-cased'
#pretrained_bert = 'emilyalsentzer/Bio_ClinicalBERT'
#pretrained_bert = 'dmis-lab/biobert-v1.1'
#pretrained_bert = 'allenai/biomed_roberta_base' 
#pretrained_bert = 'bert-large-uncased'
#pretrained_bert = 'microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract'
#pretrained_bert = 'michiyasunaga/BioLinkBERT-base'
#pretrained_bert = 'microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract'
#pretrained_bert = 'microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext'
#pretrained_bert = 'bert-base-uncased'
#pretrained_bert = 'monologg/biobert_v1.1 pubmed'

#TRAINING FILE
use_huggingface = False
training_file = "PsyTAR_JC.csv"
# training_file = "**training**.csv"
column1_AEtext = "test" #"Example"
column2_AEflag = "label" #"AE Flag"
train_test_split_percent = 0.3
balance_multiplier = 1.0
#batch_size = 3
balance_me = True
#balance_me = False

#TEST FILE
test_file = "**test**.csv"
# test_file = "**test**.csv"
test_column1_AEtext = "test"
test_column2_AEflag = "label"

In [6]:
import os 
#os.environ['WANDB_NOTEBOOK_NAME'] = 'BERTcomparison.ipynb'

In [7]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jon-l-collins. Use `wandb login --relogin` to force relogin


True

In [8]:
sweep_config = {
    'method': 'random'
    }

In [9]:
metric = {
        'name': 'val_loss',
        'goal': 'minimize'
}
sweep_config['metric'] = metric

In [10]:
import datetime
print(str(datetime.datetime.now()))

2024-03-02 02:56:06.941683


In [11]:
parameters_dict = {
    'learning_rate': {
        'values': [5e-6,5e-5]
    },
    'batch_size': {
        'values': [4,8,16]
        },
    
    'dropout': {
          'values': [0.15,0.2,0.25]
        },
    
    'epochs': {
        'values': [8]},
    
    'weight_decay': {
        'values': [0.001,0.01]
    }   
 
}
sweep_config['parameters'] = parameters_dict
pretrained_bert2 = pretrained_bert.replace('/','')
sweep_id = wandb.sweep(sweep_config, project=f"class_psytar_{pretrained_bert2}_{str(datetime.datetime.now())[:10]}")

Create sweep with ID: 16v8r0sf
Sweep URL: https://wandb.ai/jon-l-collins/class_psytar_cimm-kznendr-bert_2024-03-02/sweeps/16v8r0sf


In [12]:
#

In [13]:
import torch
#from tqdm.notebook import tqdm
#from tqdm import tqdm
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
import numpy as np
import time
import datetime
import random
import os
import pandas as pd
from tqdm.notebook import tqdm

In [14]:
from datasets import load_dataset
if use_huggingface == True:
    dataset = load_dataset("ade_corpus_v2","Ade_corpus_v2_classification")
else:
    df = pd.read_csv(training_file, encoding='unicode_escape') #changed

In [15]:
df.head()

,test,label
0,"extreme weight gain, short-term memory loss, h...",1
1,I am detoxing from Lexapro now.,0
2,I slowly cut my dosage over several months and...,0
3,I am now 10 days completely off and OMG is it ...,0
4,"I have flu-like symptoms, dizziness, major moo...",0


In [16]:
if use_huggingface == True:
    #convert to pandas dataframe
    df = pd.DataFrame(dataset['train'])

In [17]:
df[column2_AEflag].value_counts()

label
0    3841
1    2168
Name: count, dtype: int64

In [18]:
num_AEs = df[column2_AEflag].value_counts()[1]
balance_number = int(num_AEs * balance_multiplier)

# num_AEs = 3000
# balance_number = int(num_AEs * balance_multiplier)

In [19]:
print(f"num_AEs = {num_AEs}\nbalance_number = {balance_number}\nbalance_multiplier = {balance_multiplier}")

num_AEs = 2168
balance_number = 2168
balance_multiplier = 1.0


In [20]:
if(balance_me == True):
    d_NoAE = df[df[column2_AEflag]==0]
    shuffle_d_NoAE = d_NoAE.sample(frac=1,random_state=42)[:balance_number]
    d_AE = df[df[column2_AEflag]==1]
    #d_AE = df[df[column2_AEflag]==1][:3000]
    df = pd.concat([shuffle_d_NoAE,d_AE])

In [21]:
df[column2_AEflag].value_counts()

label
0    2168
1    2168
Name: count, dtype: int64

In [22]:
X = df[column1_AEtext]
y = df[column2_AEflag]

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=train_test_split_percent,stratify=y, random_state=42)

In [24]:
X_test.shape , y_test.shape

((1301,), (1301,))

In [25]:
X_val,X_test_data,y_val, y_test_data = train_test_split(X_test,y_test, test_size=0.5,stratify=y_test, random_state=42)

In [26]:
X_train.shape,X_test_data.shape, y_train.shape,y_test_data.shape, X_val.shape,y_val.shape

((3035,), (651,), (3035,), (651,), (650,), (650,))

In [27]:
#tokenizer = BertTokenizer.from_pretrained(pretrained_bert, do_lower_case=True)
#tokenizer = AutoTokenizer.from_pretrained(pretrained_bert, do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained(pretrained_bert, add_prefix_space=True, use_fast=False)

In [28]:
X_train.values

array(['I liked that it made me seem emotionally detached events happened that would normally make me burst into tears but with Cymbalta I was emotionally free.',
       "I couldn't talk without stuttering, was very confused, couldn't concentrate.",
       'I LOVE IT!', ...,
       'Doctor mentioned me being on the cymbalta for about a year.',
       'It has only been two weeks since I stopped taking the med.',
       'This drug has worked really well for my depression, and I feel is worth the side effects.'],
      dtype=object)

In [29]:
encoded_data_train = tokenizer.batch_encode_plus(X_train.values,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True,
                                                    pad_to_max_length=True,
                                                    max_length=512,
                                                    return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [30]:
encoded_data_val = tokenizer.batch_encode_plus(X_val.values,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True,
                                                    pad_to_max_length=True,
                                                    max_length=512,
                                                    return_tensors='pt')

In [31]:
encoded_data_test = tokenizer.batch_encode_plus(X_test_data.values,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True,
                                                    pad_to_max_length=True,
                                                    max_length=512,
                                                    return_tensors='pt')

In [32]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
#labels = torch.tensor(dataset['AE Flag'].replace(label_dict).values)
labels_train = torch.tensor(y_train.values)

In [33]:
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
#labels = torch.tensor(dataset['AE Flag'].replace(label_dict).values)
labels_val = torch.tensor(y_val.values)

In [34]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
#labels = torch.tensor(dataset['AE Flag'].replace(label_dict).values)
labels_test = torch.tensor(y_test_data.values)

In [35]:
from torch.utils.data import TensorDataset, random_split
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [36]:
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [37]:
def ret_dataloader():
    batch_size = wandb.config.batch_size
    print(f"batch_size = {batch_size}")
    dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
    dataloader_validation = DataLoader(dataset_val, sampler = SequentialSampler(dataset_val), batch_size=batch_size)
    #dataloader_test = DataLoader(dataset_test, sampler = SequentialSampler(dataset_test), batch_size=batch_size)
    return dataloader_train, dataloader_validation

In [39]:
def ret_model():
    
    configuration = AutoConfig.from_pretrained(pretrained_bert)
    configuration.hidden_dropout_prob = wandb.config.dropout
    configuration.attention_probs_dropout_prob = wandb.config.dropout
    configuration.num_labels = 2
    configuration.output_attentions = False
    configuration.output_hidden_states = False

    model = AutoModelForSequenceClassification.from_pretrained(pretrained_bert,config=configuration)
    return model

In [40]:
from transformers import AdamW, get_linear_schedule_with_warmup

def ret_optim(model):
    print(f"learning_rate = {wandb.config.learning_rate}")
    optimizer = AdamW(model.parameters(),
                      lr = wandb.config.learning_rate, 
                      eps = 1e-8,weight_decay = wandb.config.weight_decay)
    return optimizer

In [41]:
def ret_scheduler(optimizer, dataloader_train):
    epochs = wandb.config.epochs
    total_steps = len(dataloader_train) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)
    return scheduler

In [42]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [43]:
import random
import numpy as np
# seed_val = 17
# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)

In [44]:
from sklearn.metrics import f1_score

def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [45]:
import random
def train():
    wandb.init()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model=ret_model()
    model.to(device)
    dataloader_train, dataloader_validation = ret_dataloader()
    optimizer = ret_optim(model)
    scheduler = ret_scheduler(optimizer, dataloader_train)
    training_stats = []
    total_t0 = time.time()
    epochs = wandb.config.epochs
    for epoch_i in range(0, epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        t0 = time.time()
        total_train_loss = 0
        model.train()
        for step, batch in enumerate(dataloader_train):
            if step % 200 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(dataloader_train), elapsed))
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            loss = outputs[0]
            wandb.log({"train_batch_loss":loss.item()})
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        avg_train_loss = total_train_loss / len(dataloader_train)
        training_time = format_time(time.time() - t0)
        wandb.log({"train_loss": avg_train_loss, "epoch": epoch_i})
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))
        print("")
        print("Running Validation...")
        t0 = time.time()
        model.eval()
        total_eval_loss = 0
        predictions, true_vals = [], []
        #nb_eval_steps = 0
        for batch in dataloader_validation:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask,
                                labels=b_labels)
                loss=outputs[0] #checkleul from here downwards
                logits=outputs[1] 
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            predictions.append(logits)
            true_vals.append(label_ids)

        predictions = np.concatenate(predictions, axis=0) #added
        true_vals = np.concatenate(true_vals, axis=0) #added
        val_f1 = f1_score_func(predictions, true_vals) #added
        wandb.log({"val_f1": val_f1, "epoch": epoch_i}) #added
        print("  Average validation f1: {0:.2f}".format(val_f1))
        avg_val_loss = total_eval_loss / len(dataloader_validation)
        wandb.log({"val_loss": avg_val_loss,"epoch": epoch_i})
        print("  Average validation loss: {0:.2f}".format(avg_val_loss))

In [46]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: y8xqyf9r with config:
wandb: 	batch_size: 8
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	learning_rate: 5e-06
wandb: 	weight_decay: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cimm-kzn/endr-bert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


batch_size = 8
learning_rate = 5e-06

======== Epoch 1 / 8 ========
Training...


c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  Batch   200  of    380.    Elapsed: 0:01:07.

  Average training loss: 0.43
  Training epoch took: 0:01:49

Running Validation...
  Average validation f1: 0.85
  Average validation loss: 0.38

======== Epoch 2 / 8 ========
Training...
  Batch   200  of    380.    Elapsed: 0:00:46.

  Average training loss: 0.31
  Training epoch took: 0:01:27

Running Validation...
  Average validation f1: 0.87
  Average validation loss: 0.41

======== Epoch 3 / 8 ========
Training...
  Batch   200  of    380.    Elapsed: 0:00:47.

  Average training loss: 0.26
  Training epoch took: 0:01:28

Running Validation...
  Average validation f1: 0.87
  Average validation loss: 0.45

======== Epoch 4 / 8 ========
Training...
  Batch   200  of    380.    Elapsed: 0:00:46.

  Average training loss: 0.20
  Training epoch took: 0:01:27

Running Validation...
  Average validation f1: 0.87
  Average validation loss: 0.57

======== Epoch 5 / 8 ========
Training...
  Batch   200  of    380.    Elapsed: 0:00:46.

  Av

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cimm-kzn/endr-bert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


batch_size = 16
learning_rate = 5e-06

======== Epoch 1 / 8 ========
Training...


c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



  Average training loss: 0.51
  Training epoch took: 0:16:10

Running Validation...
  Average validation f1: 0.84
  Average validation loss: 0.39

======== Epoch 2 / 8 ========
Training...


epoch,▁▁▁
train_batch_loss,████▇▆▇▇▆▆▅▆▅▇▅▄▄▂▆▅▄▇▅▄▄▅▂▂▁▂▅▁█▅▂▃▅▄▂▂
train_loss,▁
val_f1,▁
val_loss,▁
epoch,0
train_batch_loss,0.56653
train_loss,0.51435
val_f1,0.83524
val_loss,0.38752
